In [ ]:
from tools.providers import WiktionaryProvider
from tools.dumps import download_to

WiktionaryProvider.dump_full_dataset(revision="20200701")

https://dumps.wikimedia.org/enwiktionary/20200701/enwiktionary-20200701-pages-articles.xml.bz2
[]

In [ ]:
from wiktextract import (WiktionaryConfig, parse_wiktionary, wiktionary_languages, PARTS_OF_SPEECH)
from tools.providers import WiktionaryProvider

config = WiktionaryConfig(
                    capture_languages=["English", "Translingual"],
                    capture_translations=True,
                    capture_pronunciation=True,
                    capture_compounds=True,
                    capture_redirects=True)


parse_wiktionary(path, config, word_cd, capture_cb)